In [74]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from python_module.pricing_model import HestonModel
pd.options.display.float_format = '{:.2f}'.format

In [75]:
# -------
# INPUT
# -------
S0 = 100                  # Initial stock price
K = 100                   # Strike price
T = 1.0                   # Time to maturity (1 year)
r = 0.04                  # Risk-free rate
v0 = 0.16**2              # Initial variance
theta = 0.20**2           # Long-term variance
kappa = 2.0               # Mean reversion speed
xi = 2                    # Volatility of variance
rho = -0.5                # Correlation between stock and variance
n_steps = 252             # Number of time steps (daily)
n_simulations = 1000      # Number of Monte Carlo simulations
bump = 0.01               # Bump size for delta calculation
tc = ((0.01)/100)         # Future transaction cost
margin_pct = 0.05         # Percentage of notional for maintenance margin
mumtiplier = -1           # Multiplier of the strategy
initial_notional = 100    # Initial notional

In [76]:
# Simulate prices
prices, _ = HestonModel.compute_monte_carlo(S0, K, T, r, v0, theta, kappa, xi, rho, n_simulations, n_steps)

In [77]:

results = dict()
for simulation in prices:
    df_temp = prices[simulation].to_frame('index')
    df_temp['index_pct_change'] = df_temp['index'].pct_change().fillna(0)
    df_temp['strategy_pct_change'] = df_temp['index_pct_change'].multiply(mumtiplier)
    df_temp['strategy_notional'] = df_temp['strategy_pct_change'].add(1).cumprod() * initial_notional
    df_temp['delta'] = (df_temp['strategy_notional']*bump*mumtiplier)/(df_temp['index']*bump)
    df_temp['delta_cash'] = df_temp['delta']*df_temp['index']
    df_temp['d_delta_cash'] = df_temp['delta_cash'].diff().fillna(initial_notional)
    rebalancing_cost = (df_temp['d_delta_cash'].abs()*tc).sum()
    holding_cost = ((df_temp['delta_cash'].abs()*margin_pct)*(r/360)).sum()
    total_cost = rebalancing_cost+holding_cost
    results[simulation] = total_cost/initial_notional

In [78]:
pd.Series(results).describe()*10000

count   10000000.00
mean          16.88
std            4.03
min           11.75
25%           14.95
50%           15.81
75%           17.26
max           49.74
dtype: float64